In [1]:
import os
os.environ["MUJOCO_GL"] = "egl"
os.environ["TDMPC_PATH"] = "/home/bsud/multi_task_experts/collect_myosuite/tdmpc2"
from common.parser import parse_cfg
from common.seed import set_seed
from envs import make_env
from envs.myosuite import MYOSUITE_TASKS
from tdmpc2 import TDMPC2
from pathlib import Path
from omegaconf import OmegaConf

import robomimic
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.torch_utils as TorchUtils
import robomimic.utils.tensor_utils as TensorUtils
import robomimic.utils.obs_utils as ObsUtils
import robomimic.utils.env_utils as EnvUtils
from robomimic.envs.env_base import EnvBase
from robomimic.algo import RolloutPolicy

import h5py
import json
import hydra
import numpy as np
import myosuite
import torch
from copy import deepcopy

ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /home/bsud/multi_task_experts/robomimic/robomimic/scripts/setup_macros.py
)
MyoSuite:> Registering Myo Envs


In [2]:
task_keys = list(MYOSUITE_TASKS.keys())
task_keys

['myo-reach',
 'myo-reach-hard',
 'myo-pose',
 'myo-pose-hard',
 'myo-obj-hold',
 'myo-obj-hold-hard',
 'myo-key-turn',
 'myo-key-turn-hard',
 'myo-pen-twirl',
 'myo-pen-twirl-hard']

## Add env args to dataset

In [3]:
ckpt_dir = Path("/home/bsud/multi_task_experts/robomimic/bc_trained_models/test")
data_dir = Path("/home/bsud/distill-act-datasets/")
for task in list(MYOSUITE_TASKS.keys()):
    task_name = "-".join([task.split("-")[0], "hand"] + task.split("-")[1:])
    ckpt = ckpt_dir / f"{task_name}-1.pt" 
    overrides = ["hydra/launcher=basic"]
    overrides.append(f"task={task}")
    overrides.append(f"checkpoint={str(ckpt.resolve())}")

    with hydra.initialize():
        cfg = hydra.compose("config.yaml", overrides=overrides)

    dataset_path = list(data_dir.rglob(f"{task}_*.hdf5"))[0]
    assert dataset_path.exists()
    with h5py.File(str(dataset_path), "a") as f:
        env_kwargs = OmegaConf.to_container(cfg, resolve=True)
        env_kwargs["pad_to_shape"] = (115,)
        env_args = {"env_name": MYOSUITE_TASKS[task], "type": 4, "env_kwargs": env_kwargs}
        f["data"].attrs["env_args"] = json.dumps(env_args)

/var/tmp/ipykernel_96877/3813637261.py:10: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with hydra.initialize():
/var/tmp/ipykernel_96877/3813637261.py:10: UserWarning: config_path is not specified in hydra.initialize().
See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/changes_to_hydra_main_config_path for more information.
  with hydra.initialize():
/var/tmp/ipykernel_96877/3813637261.py:10: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with hydra.initialize():
/var/tmp/ipykernel_96877/3813637261.py:10: UserWarning: config_path is not specified in hydra.initialize().
See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/changes_to_hydra_main_config_path for more information.
  with hydra.initialize():


In [4]:
with h5py.File("/home/bsud/distill-act-datasets/datasets/myo5-easy/myo-key-turn/myo-key-turn_482.hdf5") as f:
    print("attrs ", f['data'].attrs['env_args'])
    env_args = json.loads(f['data'].attrs['env_args'])

attrs  {"env_name": "myoHandKeyTurnFixed-v0", "type": 4, "env_kwargs": {"task": "myo-key-turn", "obs": "state", "num_frames": 3, "render_size": 64, "checkpoint": "/home/bsud/multi_task_experts/robomimic/bc_trained_models/myo-hand-key-turn-1.pt", "eval_episodes": 10, "eval_freq": 50000, "use_buffer": true, "hdf5_path": null, "num_episode_limit": 500, "steps": 1000000, "batch_size": 256, "reward_coef": 0.1, "value_coef": 0.1, "consistency_coef": 20, "rho": 0.5, "lr": 0.0003, "enc_lr_scale": 0.3, "grad_clip_norm": 20, "tau": 0.01, "discount_denom": 5, "discount_min": 0.95, "discount_max": 0.995, "buffer_size": 1000000, "exp_name": "default", "data_dir": "???", "mpc": true, "iterations": 6, "num_samples": 512, "num_elites": 64, "num_pi_trajs": 24, "horizon": 3, "min_std": 0.05, "max_std": 2, "temperature": 0.5, "log_std_min": -10, "log_std_max": 2, "entropy_coef": 0.0001, "num_bins": 101, "vmin": -10, "vmax": 10, "model_size": "???", "num_enc_layers": 2, "enc_dim": 256, "num_channels": 32,

In [5]:

env_args['env_kwargs']['pad_to_shape'] = None

## Add `task_id` to dataset
```python
ckpt_dir = Path("/home/krishnans/ngc/tdmpc2/ckpts/myosuite")
data_dir = Path("/home/krishnans/multi_task_experts/datasets/myo10/")
for task in list(MYOSUITE_TASKS.keys()):
    task_name = "-".join([task.split("-")[0], "hand"] + task.split("-")[1:])
    ckpt = ckpt_dir / f"{task_name}-1.pt" 
    overrides = ["hydra/launcher=basic"]
    overrides.append(f"task={task}")
    overrides.append(f"checkpoint={str(ckpt.resolve())}")

    with hydra.initialize():
        cfg = hydra.compose("config.yaml", overrides=overrides)

    dataset_path = list(data_dir.rglob(f"{task}_*.hdf5"))[0]
    assert dataset_path.exists()
    with h5py.File(str(dataset_path), "a") as f:
        print(f"writing task_id to dataset: {str(dataset_path)}") 
        for demo in f["data"].keys():
            f.create_dataset(f"data/{demo}/obs/task_id", data=np.repeat(task_keys.index(task), len(f["data"][demo]["obs"]["vec_obs"])))

```

In [6]:
env = EnvUtils.create_env_from_metadata(env_args)

Missing path to your environment variable. 
Current values LD_LIBRARY_PATH=/usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
Please add following line to .bashrc:
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/home/bsud/.mujoco/mujoco210/bin
Successfully imported MyoSuiteWrapper
class is  <class 'robomimic.envs.env_gym.EnvMyo'>
    MyoSuite: A contact-rich simulation suite for musculoskeletal motor control
        Vittorio Caggiano, Huawei Wang, Guillaume Durandau, Massimo Sartori, Vikash Kumar
        L4DC-2019 | https://sites.google.com/view/myosuite
    
Created environment with name myoHandKeyTurnFixed-v0
Action size is 39
ROBOMIMIC WARNING(
    No environment version found in dataset!
    Cannot verify if dataset and installed environment versions match
)


In [7]:
ckpt_path = "/home/bsud/multi_task_experts/robomimic/bc_trained_models/test/20240622210218/models/model_epoch_1950.pth"

policy, ckpt_dict = FileUtils.policy_from_checkpoint(
    ckpt_path=ckpt_path, 
    device='cuda:0', 
    verbose=True
)

============= Loaded Config =============
{
    "algo_name": "bc",
    "experiment": {
        "name": "test",
        "validate": false,
        "logging": {
            "terminal_output_to_txt": true,
            "log_tb": true,
            "log_wandb": false,
            "wandb_proj_name": "debug"
        },
        "mse": {
            "enabled": false,
            "every_n_epochs": 50,
            "on_save_ckpt": true,
            "num_samples": 20,
            "visualize": true
        },
        "save": {
            "enabled": true,
            "every_n_seconds": null,
            "every_n_epochs": 50,
            "epochs": [],
            "on_best_validation": false,
            "on_best_rollout_return": false,
            "on_best_rollout_success_rate": true
        },
        "epoch_every_n_steps": 100,
        "validation_epoch_every_n_steps": 10,
        "env": null,
        "additional_envs": null,
        "render": false,
        "render_video": true,
        "keep_all_v

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


============= Loaded Policy =============
ObservationKeyToModalityDict: action not found, adding action to mapping with assumed low_dim modality!
BC (
  ModuleDict(
    (policy): ActorNetwork(
        action_dim=39
  
        encoder=ObservationGroupEncoder(
            group=obs
            ObservationEncoder(
                Key(
                    name=fixed_camera
                    shape=[3, 64, 64]
                    modality=rgb
                    randomizer=ModuleList(
                      (0): None
                    )
                    net=VisualCore(
                      input_shape=[3, 64, 64]
                      output_shape=[64]
                      backbone_net=ResNet18Conv(input_channel=3, input_coord_conv=False)
                      pool_net=SpatialSoftmax(num_kp=32, temperature=1.0, noise=0.0)
                    )
                    sharing_from=None
                )
                Key(
                    name=vec_obs
                    shape=[93]
 

In [8]:

def get_obs_dict(obs, obs_keys, env, device='cuda'):
    obs_dict = {}
    if hasattr(env, 'task'):
        env_obs_dict = env.task.obs_dict
    elif hasattr(env, 'obs_dict'):
        env_obs_dict = env.obs_dict
    else:
        # handle myodex envs 
        env_obs_dict = {k: torch.as_tensor(obs[k], device=device) for k in obs_keys}

    for k in obs_keys:
        if "camera" in k and env_obs_dict[k].shape[-1] == 3:
            if len(env_obs_dict[k].shape) == 4:
                obs_dict[k] = env_obs_dict[k].permute(0, 3, 1, 2)
            else:
                obs_dict[k] = env_obs_dict[k].permute(2, 0, 1)[None]
        else:
            if len(env_obs_dict[k].shape) == 1:
                obs_dict[k] = env_obs_dict[k][None]
            else:
                obs_dict[k] = env_obs_dict[k]
            
    return obs_dict
            
def rollout(policy, env, horizon, render=False, video_writer=None, video_skip=5, camera_names=None, device='cuda'):
    """
    Helper function to carry out rollouts. Supports on-screen rendering, off-screen rendering to a video, 
    and returns the rollout trajectory.
    Args:
        policy (instance of RolloutPolicy): policy loaded from a checkpoint
        env (instance of EnvBase): env loaded from a checkpoint or demonstration metadata
        horizon (int): maximum horizon for the rollout
        render (bool): whether to render rollout on-screen
        video_writer (imageio writer): if provided, use to write rollout to video
        video_skip (int): how often to write video frames
        camera_names (list): determines which camera(s) are used for rendering. Pass more than
            one to output a video with multiple camera views concatenated horizontally.
    Returns:
        stats (dict): some statistics for the rollout - such as return, horizon, and task success
    """
    # assert isinstance(env, EnvBase)
    assert isinstance(policy, RolloutPolicy)
    assert not (render and (video_writer is not None))

    policy.start_episode()
    obs = env.reset()
    obs_keys = set(policy.policy.nets.policy.obs_shapes.keys())
    # state_dict = env.get_state()

    # hack that is necessary for robosuite tasks for deterministic action playback
    # obs = env.reset_to(state_dict)

    results = {}
    video_count = 0  # video frame counter
    total_reward = 0.
    try:
        for step_i in range(horizon):
            obs_dict = get_obs_dict(obs, obs_keys, env, device=device)
            print("got obs dict")
            # get action from policy
            act = policy(ob=obs_dict, batched=True)
            print("ran policy")

            # play action
            next_obs, r, done, info = env.step(act[0])

            # compute reward
            total_reward += r
            success = info["success"] # env.is_success()["task"]

            # visualization
            if video_writer is not None:
                if video_count % video_skip == 0:
                    video_img = []
                    for cam_name in camera_names:
                        video_img.append(obs_dict[cam_name].cpu().numpy()[0])
                    video_img = np.concatenate(video_img, axis=1) # concatenate horizontally
                    video_writer.append_data(video_img)
                video_count += 1

            # break if done or if success
            if done or success:
                break

            # update for next iter
            obs = deepcopy(next_obs)
            # state_dict = env.get_state()

    except env.rollout_exceptions as e:
        print("WARNING: got rollout exception {}".format(e))

    stats = dict(Return=total_reward, Horizon=(step_i + 1), Success_Rate=float(success))

    return stats


In [ ]:
# env.rollout_exceptions = ()
stats = rollout(
    policy=policy, 
    env=env, 
    horizon=100, 
    render=False, 
    video_writer=None, 
    video_skip=5, 
    camera_names=["hand_camera"]
)
print(stats)
# video_writer.close()

got obs dict
prepared obs
policy is  BC (
  ModuleDict(
    (policy): ActorNetwork(
        action_dim=39
  
        encoder=ObservationGroupEncoder(
            group=obs
            ObservationEncoder(
                Key(
                    name=fixed_camera
                    shape=[3, 64, 64]
                    modality=rgb
                    randomizer=ModuleList(
                      (0): None
                    )
                    net=VisualCore(
                      input_shape=[3, 64, 64]
                      output_shape=[64]
                      backbone_net=ResNet18Conv(input_channel=3, input_coord_conv=False)
                      pool_net=SpatialSoftmax(num_kp=32, temperature=1.0, noise=0.0)
                    )
                    sharing_from=None
                )
                Key(
                    name=vec_obs
                    shape=[93]
                    modality=low_dim
                    randomizer=ModuleList(
                      (0): Non